In [25]:
import gym
import gym_anytrading

from gym_anytrading.envs import TradingEnv, ForexEnv, StocksEnv, Actions, Positions 
from gym_anytrading.datasets import FOREX_EURUSD_1H_ASK, STOCKS_GOOGL
import matplotlib.pyplot as plt
from keras import applications
from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Dropout, Input, Concatenate
from keras.optimizers import Adam

from rl.agents.dqn import DQNAgent
from rl.agents.ddpg import DDPGAgent
from rl.policy import BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor
from rl.random import OrnsteinUhlenbeckProcess
from rl.callbacks import FileLogger, ModelIntervalCheckpoint, TrainIntervalLogger

from sklearn import preprocessing
import warnings
warnings.filterwarnings('ignore')

import json
import numpy as np

In [26]:
class EnvProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 2  # (height, width, channel)
        return self.process_obs_1(observation)
    
    def process_obs_1(self, observation):
        prices = []
        diff = []
        for o in observation:
            prices.append(o[0])
            diff.append(o[1])
        prices = preprocessing.normalize([prices], norm='l2', axis=1, copy=True, return_norm=False)[0]
        diff = preprocessing.normalize([diff], norm='l2', axis=1, copy=True, return_norm=False)[0]
        new_obs = np.column_stack((prices, diff))
        return new_obs
    
    def process_obs_2(self, observation):
        prices = []
        diff = []
        for o in observation:
            prices.append(o[0])
            diff.append(o[1] / o[0] * 100)
        prices = preprocessing.normalize([prices], norm='l2', axis=1, copy=True, return_norm=False)[0]
        new_obs = np.column_stack((prices, diff))
        return new_obs
        
    def process_state_batch(self, batch):
#         print(batch)
        return batch

    def process_reward(self, reward):
        return reward

class DDPGEnvProcessor(EnvProcessor):
    def process_action(self,action):
        return np.argmax(action)
    
class CustomEpsGreedyQPolicy(EpsGreedyQPolicy):
    def __init__(self, eps=0.1, update_interval=100):
        EpsGreedyQPolicy.__init__(self, eps)
        self.update_interval = update_interval
        self.count = 0
        self.init_eps = self.eps
        
    def select_action(self, q_values):
        assert q_values.ndim == 1
        nb_actions = q_values.shape[0]

        if np.random.uniform() < self.eps:
            action = np.random.randint(0, nb_actions)
        else:
            action = np.argmax(q_values)
        self.count += 1
        if (self.count % self.update_interval) == 0:
            self.eps = self.init_eps / (self.count / self.update_interval)
            print(self.eps)
        return action

In [27]:
env = gym.make('forex-v0', frame_bound=(50, 4000), window_size=10)
nb_actions = env.action_space.n

# Next, we build a very simple model.
model = Sequential()
model.add(Flatten(input_shape=(1,) + env.observation_space.shape))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(1024, activation='relu'))
model.add(Dense(nb_actions, activation='linear'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten_7 (Flatten)          (None, 20)                0         
_________________________________________________________________
dense_22 (Dense)             (None, 1024)              21504     
_________________________________________________________________
dense_23 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_24 (Dense)             (None, 1024)              1049600   
_________________________________________________________________
dense_25 (Dense)             (None, 2)                 2050      
Total params: 2,122,754
Trainable params: 2,122,754
Non-trainable params: 0
_________________________________________________________________
None


In [28]:
# 3 = enable_double_dqn=True, nb_steps=1000000, EpsGreedyQPolicy **
# 4 = enable_double_dqn=True, nb_steps=100000, EpsGreedyQPolicy
# 5 = enable_double_dqn=True, nb_steps=100000, BoltzmannQPolicy **
# 6 = enable_double_dqn=True, nb_steps=1000000, CustomEpsGreedyQPolicy, process_obs_2
# 7 = enable_double_dqn=True, nb_steps=1000000, EpsGreedyQPolicy, process_obs_2 **
# 8 = enable_double_dqn=True, nb_steps=1000000, CustomEpsGreedyQPolicy **
# 9 = enable_double_dqn=True, nb_steps=1000000, CustomEpsGreedyQPolicy(update_interval=2500), train_interval=256, batch_size=512
# 10 = enable_double_dqn=True, nb_steps=1000000, BoltzmannQPolicy, train_interval=256, batch_size=512 **
# 11 = DDPG, train_interval=256, batch_size=512 ***
# 12 = enable_double_dqn=True, nb_steps=1000000, BoltzmannQPolicy, train_interval=32, batch_size=64 
# 13 = DDPG, train_interval=128, batch_size=64 ***
# 14 = DDPG, train_interval=128, batch_size=256, lr = 0.01, target_model_update=0.01, gamma=0.9
# 15 = DDPG, train_interval=128, batch_size=256, lr = 0.0001, target_model_update=0.0001, gamma=0.9
train_no = 14
weights_filename = 'dqn_weights_{}.h5f'.format(train_no)
checkpoint_weights_filename = 'dqn_weights_{step}_'+'{}.h5f'.format(train_no)
log_filename = 'dqn_log_{}.csv'.format(train_no)
callbacks = [ModelIntervalCheckpoint(checkpoint_weights_filename, interval=680)]
callbacks += [TrainIntervalLogger(interval=1)]

In [29]:
processor = EnvProcessor()
memory = SequentialMemory(limit=100000, window_length=1)
# policy = CustomEpsGreedyQPolicy(update_interval = 2500, eps = 1.0)
policy = BoltzmannQPolicy()
agent = DQNAgent(model=model, nb_actions=nb_actions, memory=memory, nb_steps_warmup=20,
               target_model_update=1e-2, policy=policy, batch_size=64, processor=processor, 
               train_interval=32, enable_double_dqn=True)
agent.compile(Adam(lr=1e-3), metrics=['mae'])


In [30]:
agent.fit(env, nb_steps=4000, visualize=False, verbose=2, callbacks=callbacks)

# After training is done, we save the final weights.
agent.save_weights('dqn_{}_weights.h5f'.format('forex-v0'), overwrite=True)

Training for 2000 steps ...
Training for 2000 steps ...
Interval 1 (0 steps performed)
1/1 [==============================] - 0s 201ms/step - reward: 24.3000
Interval 2 (1 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 12.3000
Interval 3 (2 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.3000
Interval 4 (3 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: -2.8000
Interval 5 (4 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 6 (5 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: -3.6000
Interval 7 (6 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 11.0000
Interval 8 (7 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 9 (8 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: -21.2000
Interval 10 (9 steps performed)
1/1 

Interval 80 (79 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 81 (80 steps performed)
1/1 [==============================] - 0s 4ms/step - reward: 0.0000e+00
Interval 82 (81 steps performed)
1/1 [==============================] - 0s 4ms/step - reward: 0.0000e+00
Interval 83 (82 steps performed)
1/1 [==============================] - 0s 4ms/step - reward: 0.0000e+00
Interval 84 (83 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 85 (84 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 86 (85 steps performed)
1/1 [==============================] - 0s 4ms/step - reward: 0.0000e+00
Interval 87 (86 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 88 (87 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 89 (88 steps performed)
1/1 [=================

Interval 158 (157 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 159 (158 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 160 (159 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 161 (160 steps performed)
1/1 [==============================] - 0s 51ms/step - reward: 0.0000e+00
Interval 162 (161 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 163 (162 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 164 (163 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 165 (164 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 166 (165 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 167 (166 steps performed)
1

Interval 235 (234 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 236 (235 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 237 (236 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 238 (237 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 239 (238 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 240 (239 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 241 (240 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 242 (241 steps performed)
1/1 [==============================] - 0s 4ms/step - reward: 0.0000e+00
Interval 243 (242 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 244 (243 steps performed)
1/

Interval 312 (311 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 313 (312 steps performed)
1/1 [==============================] - 0s 4ms/step - reward: 0.0000e+00
Interval 314 (313 steps performed)
1/1 [==============================] - 0s 4ms/step - reward: 0.0000e+00
Interval 315 (314 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 316 (315 steps performed)
1/1 [==============================] - 0s 4ms/step - reward: 0.0000e+00
Interval 317 (316 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 318 (317 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 319 (318 steps performed)
1/1 [==============================] - 0s 5ms/step - reward: 0.0000e+00
Interval 320 (319 steps performed)
1/1 [==============================] - 0s 4ms/step - reward: 0.0000e+00
Interval 321 (320 steps performed)
1/

...

Interval 1692 (1691 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 13.6000
Interval 1693 (1692 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 1694 (1693 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 8.1000
Interval 1695 (1694 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 1696 (1695 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 1697 (1696 steps performed)
1/1 [==============================] - 0s 50ms/step - reward: 0.0000e+00
Interval 1698 (1697 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 1699 (1698 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 14.4000
Interval 1700 (1699 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 1701 (1700 steps pe

Interval 1768 (1767 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 1769 (1768 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 1770 (1769 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 1771 (1770 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: -16.4000
Interval 1772 (1771 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: -2.3000
Interval 1773 (1772 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 1774 (1773 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 1775 (1774 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 1776 (1775 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 1777 (1776 step

Interval 1844 (1843 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 39.1000
Interval 1845 (1844 steps performed)
1/1 [==============================] - 0s 4ms/step - reward: 0.0000e+00
Interval 1846 (1845 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: -2.4000
Interval 1847 (1846 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 1848 (1847 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 1849 (1848 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 1850 (1849 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 1851 (1850 steps performed)
1/1 [==============================] - 0s 4ms/step - reward: 0.0000e+00
Interval 1852 (1851 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: -1.2000
Interval 1853 (1852 steps pe

Interval 1920 (1919 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: -9.6000
Interval 1921 (1920 steps performed)
1/1 [==============================] - 0s 50ms/step - reward: 0.0000e+00
Interval 1922 (1921 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 1923 (1922 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 0.0000e+00
Interval 1924 (1923 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: -8.7000
Interval 1925 (1924 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 4.0000
Interval 1926 (1925 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 1927 (1926 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: 15.5000
Interval 1928 (1927 steps performed)
1/1 [==============================] - 0s 3ms/step - reward: -11.3000
Interval 1929 (1928 steps perform

Interval 1997 (1996 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 1998 (1997 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: -6.2000
Interval 1999 (1998 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
Interval 2000 (1999 steps performed)
1/1 [==============================] - 0s 2ms/step - reward: 0.0000e+00
done, took 12.367 seconds
done, took 12.367 seconds


In [31]:
env._total_reward

-229.40000000003016

In [32]:
env._total_profit

0.9081862534457926